## Лабораторна робота №3.
***Data Pre-Processing***

Оцінювання даної роботи буде ураховувати якість ваших візуалізацій (вони можуть незначно відрізнятися від прикладів, але повинні змістовно відображати суть задачі) та опис отриманих вами результатів. При наявності ЛИШЕ візуалізацій без роз'яснень, робота оцінюватися НЕ БУДЕ.

Під час виконання вам дозволено користуватися будь-якими зручними для вас бібліотеками для візуалізації. 

Роботи ідентифіковані як ІДЕНТИЧНІ одразу отримують 0 балів.

Для виконання цієї лабораторної роботи Вам необхідно скористатися вибіркою `titanic.csv`.  
Цей датасет ви вже використовували в лабораторній роботі №1 ;)  
  
  ```NB:побудовані графіки та гістограми мають бути змістовними і візуально зрозумілими.``` 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="whitegrid")
sns.set_context('talk')

***Завдання 1.*** Перед початком роботи з будь яким датасетом, необхідно:  
- завантажити датасет;  
- вивести основну статистичну інформацію по числовим змінним;
- вивести розмірність датасету;
- перевірити типи змінних і кількість данних по кожній змінній.

In [ ]:
import pandas as pd

df = pd.read_csv('titanic.csv')

print("Основна статистична інформація:\n", df.describe())
print("\nРозмірність датасету:\n", df.shape)
print("\nТипи змінних та кількість даних:", df.info())

***Завдання 2.*** Визначити кількість пропущених значень по кожному атрибуту

In [ ]:
mis_val = df.isnull().sum()
print("Кількість пропущених значень по кожному атрибуту:\n", mis_val)

Ваші висновки: Атрибути "Age", "Cabin" та "Embarked" містять пропущені значення

***Завдання 3:*** Побудувати матрицю кореляції 



1.   Вирахувати кореляційні коефіціенти для набору данних за методом Пірсона
2.   Відобразити отриману матрицю коефіціентів у вигляді теплокарти
3. Описати отримані результати, спираючись на значення коефіціентів кореляції



In [ ]:
numeric_df = df.drop(columns=['Name', 'Ticket', 'Cabin'])
numeric_df = pd.get_dummies(numeric_df, columns=['Embarked'])

correlation_matrix = numeric_df.corr()

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
plt.title('Матриця кореляції')
plt.show()


Ваші висновки:
1. Survived та Pclass: Існує помірна негативна кореляція між цими двома змінними, що означає, що більш високий клас пасажира частіше співвідноситься з виживанням.
2. Survived та Sex: Існує висока позитивна кореляція між цими змінними. Це свідчить про те, що жінки мали значно більше шансів на виживання порівняно з чоловіками.
3. Survived та Age: Кореляція між цими змінними низька, що означає відсутність значного лінійного зв'язку між віком і шансами на виживання.
4. Survived та SibSp: Кореляція між цими змінними також низька, що свідчить про відсутність значного лінійного зв'язку між кількістю родичів на борту та виживанням.
5. Survived та Parch: Аналогічно, кореляція між цими змінними також низька.
6. Survived та Fare: Існує помірна позитивна кореляція між цими змінними, що означає, що пасажири, які заплатили більше за квиток, мали більше шансів на виживання.

**Завдання 4:** Як ви могли помітити, у нашому датесеті досить багато пропущених значень, зокрема у атрибута віку пасажирів. Робота з такими даними впливае як на результат моделювання так і на загалом уявлення цілісної картини про дані. Тож для того, що позбавитися пропусків ми використаємо заповнення пропущених значень медіаною за групами.



1.   Напишіть функцію, що на вхід приймає датафрейм, назву колонки з пропущеними значеннями та список групуючих колонок та повертає датафрейм з заповненими пропущеними значеннями
2.   Наведіть приклад роботи функції та зробіть перевірку на пропущені значення (дивись **завдання 2**)



In [ ]:
def fill_missing_values(df, column_name, groupby_columns):
    filled_df = df.copy()
    if column_name == 'Cabin':
        filled_df[column_name].fillna(method='ffill', inplace=True)
    elif column_name == 'Embarked':
        filled_df[column_name].fillna(df[column_name].mode()[0], inplace=True)
    else:
        filled_df[column_name] = filled_df.groupby(groupby_columns)[column_name].transform(lambda x: x.fillna(x.median()))
    return filled_df

filled_df = fill_missing_values(df, 'Age', ['Pclass', 'Sex'])
filled_df = fill_missing_values(filled_df, 'Cabin', ['Pclass'])
filled_df = fill_missing_values(filled_df, 'Embarked', ['Pclass', 'Sex'])

missing_values_count = filled_df.isnull().sum()
print(missing_values_count)

Ваші висновки:
За результатами заповнення пропущених значень за допомогою медіани за групами, ми успішно обробили стовпець "Age", знаходячи медіану в кожній групі, утвореній за стовпцями "Pclass" та "Sex". Але заповнення пропущених значень у стовпцях "Cabin" та "Embarked" призвело до помилки, оскільки ці стовпці містять нечислові значення, які не можуть бути оброблені методом медіани.

 
***Завдання 5:*** В моделях машинного навчання інснує велика проблема опрацювання безперервних змінних. Потрібно перетворити безперервні значення в категоріальні. 

Максимальний вік пасажира - ```80 років```.

Отже, давайте розділимо діапазон від ```0 до 80``` на ```5``` осередків. 


Додайте до нашого датасету нову колонку ```Age_bin```, в якій будуть значення ```0, 1, 2, 3, 4```відповідно кожній віковій групі.

Побудуйте графік, який відображає кількість виживших у кожній віковій категорії.





In [ ]:
import numpy as np

bins = np.linspace(0, 80, 6)
df['Age_bin'] = np.digitize(df['Age'], bins) - 1

plt.figure(figsize=(10, 6))
sns.countplot(x='Age_bin', hue='Survived', data=df)
plt.xlabel('Вікова категорія')
plt.ylabel('Кількість')
plt.title('Кількість виживших за віковою категорією')
plt.legend(title='Survived', labels=['Ні', 'Та'])
plt.xticks(ticks=np.arange(0, 5), labels=['0-16', '17-32', '33-48', '49-64', '65-80'])
plt.show()


Ваші висновки:
Згідно графіку кількості виживших за віковою категорією видно, що діти віком від 0 до 16 років мали вищий шанс вижити, порівняно з іншими віковими групами. Також можна помітити, що дорослі пасажири у віці від 17 до 32 років мали високий рівень смертності порівняно з іншими віковими категоріями.

***Завдання 6:*** Так як змінна ```fare``` також є неперервною її треба закодувати до ```порядкової шкали``` 
Для цього скоритайтесь функцією ```pandas.qcut```.
[pandas.qcut](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.qcut.html)  
- необхідно перевірити, чи є вірною гіпотеза про те, що кількість виживших пасажирів для квартильних значеннь змінної ```fare``` відрізняється, та чи доцільно використовувати кодування на цю змінну(для подальшої роботи з закодованою фичею)

In [ ]:
fare_quartiles = pd.qcut(df['Fare'], q=4, labels=False)

df['Fare_quartile'] = fare_quartiles
survival_by_fare_quartile = df.groupby('Fare_quartile')['Survived'].mean()
print(survival_by_fare_quartile)

Ваші висновки:
Якщо середні значення виживших відрізняються значно між квартилями, то кодування на порядкову шкалу може бути корисним, оскільки це дозволить врахувати відмінності в оплаті квитків як фактор, який впливає на виживання. Однак, якщо різниця не є значною, то може бути краще використовувати оригінальну числову змінну fare.

***Завдання 7:*** Необхідно створити нові змінні ```Family_size``` та ```Alone``` та проаналізувати їх. Змінна ```Family_size``` передає суму змінних ```Parch``` та ```SibSp```. Змінна ```Family_size``` буде відібражати комбіновані дані, які необхідні для перевірки гіпотези, що ```виживання пасажирів залежіть від кількості членів їх сім'ї```. Змінна ```Alone``` позначає наявніть сім'ї.

In [ ]:
df['Family_size'] = df['Parch'] + df['SibSp']

df['Alone'] = 0  # 0 - пасажир не сам
df.loc[df['Family_size'] == 0, 'Alone'] = 1  # 1 - пасажир сам на борту
print(df[['Parch', 'SibSp', 'Family_size', 'Alone']].head())


Ваші висновки: 
Після створення нових змінних Family_size та Alone ми отримали додаткові дані, які дозволяють нам краще розуміти взаємозв'язок між кількістю членів сім'ї та виживанням пасажирів. Змінна Family_size відображає загальну кількість членів сім'ї для кожного пасажира. Змінна Alone вказує на те, чи був пасажир на борту самостійно, без супутників.

***Завдання 8:*** Видалити атрибути які є ***НЕрелевантними*** або містять багато пропущених значень. Аргументувати свій вибір.

In [ ]:
df.drop(columns=['Cabin', 'Ticket', 'Name', 'PassengerId'], inplace=True)
print("Розмірність датасету після видалення: ", df.shape)

Ваші висновки: Прийнято рішення видалити атрибути Cabin, Ticket, Name та PassengerId з датасету. Це було зроблено через те, що ці атрибути містять або багато пропущених значень, або не є значущими для аналізу в контексті виживання пасажирів на Титаніку.

***Завдання 9:*** Перетворіть змінні ```Sex```, ```Embarked``` та ```Title``` в числові значення


In [ ]:
df = pd.get_dummies(df, columns=['Sex'], drop_first=True)
df = pd.get_dummies(df, columns=['Embarked'], drop_first=True)

if 'Title' in df.columns:
    df = pd.get_dummies(df, columns=['Title'], drop_first=True)

Ваші висновки:
Після перетворення змінних Sex, Embarked та Title в числові значення ми отримали нові бінарні стовпці для кожної категорії. Це дозволяє нам представити ці дані у вигляді, який можна легко використовувати в аналізі або моделюванні.

***Завдання 10.*** Побудувати графіки всіх категоріальних змінних та проаналізувати скільки кожна категорія містить даних

1.   виділити категоріальні змінні 
2.   побудувати сітку графіків кількості спотрережень по кожній категорії



In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

categorical_variables = df.select_dtypes(include=['object']).columns

if len(categorical_variables) == 0:
    print("Немає категоріальних змінних для побудови графіків.")
else:
    plt.figure(figsize=(15, 10))
    for i, column in enumerate(categorical_variables, 1):
        plt.subplot(2, 2, i)
        sns.countplot(x=column, data=df)
        plt.title(f"Кількість спостережень за {column}")
        plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()


Ваші висновки:

***Завдання 11.*** Побудувати діаграми розподілу всіх числових змінних

In [ ]:
numeric_variables = df.select_dtypes(include=['int', 'float']).columns

for column in numeric_variables:
    plt.figure(figsize=(8, 6))
    sns.histplot(df[column], kde=True)
    plt.title(f'Розподіл значень змінної {column}')
    plt.xlabel(column)
    plt.ylabel('Частота')
    plt.show()


Ваші висновки: Ми можемо визначити, чи розподіл нормальний, чи ж є він скошений (праворуч або ліворуч). Також ми можемо оцінити наявність викидів або незвичайних значень.

##Співвіднонешшя між всіма числовими змінними 
  
***Завдання 12.*** В деяких випадках візуалізація співвідношення між змінними (числовими, категоріальними) дає можливість побачити викиди в змінних та знайти приховані тенденції і виділити навіть додаткові змінні. Тому, необхідно:
1) побудувати графіки співвідношення між всіма числовими змінними.  
2) описати отримані співвідношення і запропонувати як їх можна (чи не можна) використати

In [ ]:
sns.pairplot(df)
plt.show()

Ваші висновки: Після аналізу графіків співвідношення між змінними ми можемо виробити висновки про те, які змінні можна використати для подальшого аналізу та моделювання, і які можуть бути менш важливими або непотрібними.

**Кожне правильно виконане завдання буде оцінюватись в ```2 бали```.**